<a href="https://colab.research.google.com/github/nadavru/iLearn/blob/iLearnML/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Imports and Base Classes

In [9]:

import numpy as np
from abc import abstractmethod
import math

In [10]:
class Module():

    @abstractmethod
    def forward(self, x):
        pass
    
    @abstractmethod
    def backward(self, loss):
        pass
        
    @abstractmethod
    def train(sel):
        pass
    
class Loss():

    @abstractmethod
    def forward(self, x, y):
        pass
    
    @abstractmethod
    def backward(self):
        pass

# NN Blocks and Loss

In [11]:
class Tanh(Module):

    def forward(self, x: np.array):
        self.temp = (np.exp(x) + np.exp(-x))
        return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
    
    def backward(self, loss: np.array):
        return 4*loss/np.square(self.temp)
    
    def train(self):
        pass

class ReLU(Module):

    def forward(self, x: np.array):
        self.tmp = (x>0)
        return x * self.tmp
    
    def backward(self, loss: np.array):
        return self.tmp * loss
    
    def train(self):
        pass

class Sigmoid(Module):

    def forward(self, x: np.array):
        self.tmp = np.exp(x)
        return self.tmp /(self.tmp + 1)
    
    def backward(self, loss: np.array):
        return self.tmp / np.square(self.tmp + 1) * loss
    
    def train(self):
        pass

class Linear(Module):

    def __init__(self, in_dim: int, out_dim: int, lr: float = 0.01) -> None:
        self.weights = np.random.rand(in_dim, out_dim)
        self.biases = np.random.rand(1, out_dim)
        self.lr = lr

    def forward(self, x: np.array):
        self.x = x.copy()
        return x @ self.weights + self.biases
    
    def backward(self, loss: np.array):
        self.grad_w = self.x.transpose() @ loss
        self.grad_b = np.sum(loss, axis=0, keepdims=True)
        return loss @ self.weights.transpose()
        
    def train(self):
        self.weights -= self.lr*self.grad_w
        self.biases -= self.lr*self.grad_b

class Sequential(Module):

    def __init__(self, hidden_dims: list, activation, lr: float) -> None:
        layers = []
        input_dim = 2
        for dim in hidden_dims:
            layers.append(Linear(input_dim, dim, lr))
            if activation is not None:
                layers.append(activation())
            input_dim = dim
        layers.append(Linear(input_dim, 1, lr))
        self.layers = layers
    
    def forward(self, x: np.array):
        input = x
        for layer in self.layers:
            input = layer.forward(input)
        return input
    
    def backward(self, loss: np.array):
        input = loss
        for layer in reversed(self.layers):
            input = layer.backward(input)
        return input
        
    def train(self):
        for layer in self.layers:
            layer.train()

In [12]:
class MSELoss(Loss):

    def forward(self, x: np.array, y: np.array):
        self.x = x.copy()
        self.y = y.copy()
        return np.mean(np.square(x-y), axis=0)[0]
    
    def backward(self):
        return 2 * (self.x-self.y) / self.x.shape[0]

# Function Calculation

In [13]:
'''def f(x: np.array, y: np.array):

    return (x * np.exp(-(x**2+y**2))).reshape((-1,1))'''

class Function():

    def __init__(self, string) -> None:
        if len([letter for letter in string.replace("pi","").replace("e","").replace("x","").replace("y","").replace("r","") if letter.isalpha()]):
            print("Abort.")
            exit()
        self.string = string.replace("^","**").replace("e","math.e").replace("pi","math.pi")

    
    def forward(self, x: np.array, y: np.array):
        if self.string.find("r")!=-1:
            r = lambda x: x * (x > 0)
        return eval(self.string).reshape((-1,1))

# NN building and training

In [22]:
class NN():

    def __init__(self, hidden_dims: list, activation, lr: float, f_string: str, epochs: int, batch_size: int) -> None:
        self.model = Sequential(hidden_dims, activation, lr)
        self.criterion = MSELoss()
        self.F = Function(f_string)
        X = np.arange(-2, 2, 0.2)
        Y = np.arange(-2, 2, 0.2)
        X, Y = np.meshgrid(X, Y)
        self.grid_points = np.concatenate((X.reshape((-1,1)), Y.reshape((-1,1))), axis=-1)
        self.batch = np.random.rand(batch_size, 2) * 4 - 2
        self.epochs = epochs
        url = "https://i-learn-ml.oa.r.appspot.com/viewer/viewerGD.html#"
        self.disp_url = url+f"xmax=2.1&xmin=-2.1&ymax=2.1&ymin=-2.1&func={f_string}&points="
    
    def add_points(self):
        predict = self.model.forward(self.grid_points)
        predict = (predict*1000).astype(int)/1000
        for i in range(predict.shape[0]):
            self.disp_url += f"({self.grid_points[i,0]},{self.grid_points[i,1]},{predict[i,0]})"
    
    def train(self):
        self.add_points()
        for _ in range(self.epochs):
            predict = self.model.forward(self.batch)
            loss = self.criterion.forward(predict, self.F.forward(self.batch[:,0], self.batch[:,1]))
            self.model.backward(self.criterion.backward())
            self.model.train()
            self.disp_url += "|"
            self.add_points()
        return self.disp_url

In [25]:
nn = NN([3,5], ReLU, 0.05, "x*e*pi^(-x^2-y^2)", 50, 100)
src = nn.train()
print(len(src))

876492


In [26]:
'''nn = Sequential([3,5], ReLU, 0.05)
criterion = MSELoss()
# F = Calc("x*e*pi^(-x^2-y^2)")
F = Function("r(y)")
max_grid = 2
x = np.random.rand(100,2) * 2 * max_grid - max_grid
loss = 1
steps = 0
while loss>0.002 and steps<150:
    predict = nn.forward(x)
    loss = criterion.forward(predict, F.forward(x[:,0], x[:,1]))
    nn.backward(criterion.backward())
    nn.train()
    steps += 1
print(f"loss: {loss}")
print(f"steps: {steps}")
print(nn.forward(np.array([[1,2],[2,10],[3,5],[7,-10],[21,33]])))'''

'nn = Sequential([3,5], ReLU, 0.05)\ncriterion = MSELoss()\n# F = Calc("x*e*pi^(-x^2-y^2)")\nF = Function("r(y)")\nmax_grid = 2\nx = np.random.rand(100,2) * 2 * max_grid - max_grid\nloss = 1\nsteps = 0\nwhile loss>0.002 and steps<150:\n    predict = nn.forward(x)\n    loss = criterion.forward(predict, F.forward(x[:,0], x[:,1]))\n    nn.backward(criterion.backward())\n    nn.train()\n    steps += 1\nprint(f"loss: {loss}")\nprint(f"steps: {steps}")\nprint(nn.forward(np.array([[1,2],[2,10],[3,5],[7,-10],[21,33]])))'